In [2]:
import pandas as pd
import numpy as np

In [4]:
product_cols = ['ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1',
                'ind_cno_fin_ult1','ind_ctju_fin_ult1','ind_ctma_fin_ult1','ind_ctop_fin_ult1',
                'ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_dela_fin_ult1','ind_deme_fin_ult1',
                'ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_nom_pens_ult1',
                'ind_nomina_ult1','ind_plan_fin_ult1','ind_pres_fin_ult1','ind_reca_fin_ult1',
                'ind_recibo_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1']

In [5]:
train = pd.read_csv(r'E:\Santander Recommendation\train_ver2.csv')

C:\Users\liamc\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
#get the weights for the month of june
df_jun15 = train.loc[train['fecha_dato']=='2015-06-28',product_cols+['ncodpers']]
df_may15 = train.loc[train['fecha_dato']=='2015-05-28', product_cols+['ncodpers']]
train = train.loc[train['fecha_dato']=='2016-05-28']

In [7]:
dfm = pd.merge(df_jun15,df_may15, how='left', on=['ncodpers'], suffixes=('', '_5'))

In [8]:
prevcols = [col for col in dfm.columns if '_ult1_'+str(5) in col]
currcols = [col for col in dfm.columns if '_ult1' == col[-5:]]

In [9]:
all_product_col = [col for col in dfm.columns if '_ult1' in col]

for col in all_product_col:
    dfm[col].fillna(0, inplace=True)

In [10]:
for col in currcols:
    dfm[col] = dfm[col] - dfm[col+'_'+str(5)]
    dfm[col] = dfm[col].apply(lambda x: max(x,0))

In [11]:
dfm = dfm.drop('ncodpers',axis=1)
dfm = dfm[currcols]

In [12]:
sums = []
for col in dfm.columns:
    sums.append(dfm[col].sum())

In [13]:
weights = []
total = sum(sums)
for i in sums:
    weights.append(i/total)

In [33]:
lastmonthlabels = train[product_cols]
lastmonthlabels['ncodpers'] = train.ncodpers #check if another last month
lastmonthlabels = lastmonthlabels.set_index('ncodpers')
stacked_lastmonthlabels = lastmonthlabels.stack()
filtered_lastmonthlabels = stacked_lastmonthlabels.reset_index()
filtered_lastmonthlabels.columns = ["ncodpers", "product", "is_owned"]
lastmonthlabels=[]

C:\Users\liamc\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [34]:
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import accuracy
from surprise.model_selection import train_test_split
reader = Reader(line_format='user item rating', rating_scale=(0,1))
data = Dataset.load_from_df(filtered_lastmonthlabels, reader=reader)
#trainset, testset = train_test_split(data, test_size=.05)
trainset = data.build_full_trainset()
algo = SVD(n_factors=50,verbose=True)
algo.fit(trainset)
#predictions = algo.test(testset)
#print(accuracy.rmse(predictions))

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


In [3]:
test = pd.read_csv(r'E:\Santander Recommendation\test_ver2.csv')

C:\Users\liamc\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
test.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento
0,2016-06-28,15889,F,ES,V,56,1995-01-16,0,256,1,...,N,N,KAT,N,1,28.0,MADRID,1,326124.90,01 - TOP
1,2016-06-28,1170544,N,ES,H,36,2013-08-28,0,34,1,...,N,NaN,KAT,N,1,3.0,ALICANTE,0,NA,02 - PARTICULARES
2,2016-06-28,1170545,N,ES,V,22,2013-08-28,0,34,1,...,N,NaN,KHE,N,1,15.0,"CORUÑA, A",1,NA,03 - UNIVERSITARIO
3,2016-06-28,1170547,N,ES,H,22,2013-08-28,0,34,1,...,N,NaN,KHE,N,1,8.0,BARCELONA,0,148402.98,03 - UNIVERSITARIO
4,2016-06-28,1170548,N,ES,H,22,2013-08-28,0,34,1,...,N,NaN,KHE,N,1,7.0,"BALEARS, ILLES",0,106885.80,03 - UNIVERSITARIO


In [15]:
users = test.ncodpers

In [16]:
uid = users[0]
iid = product_cols[2]

In [35]:
pred =algo.predict(uid,iid)

In [36]:
pred[3]

0.7173796053259962

### temp

In [17]:
#Trying using just the weights of new products from june 2015
preds =[]
user_preds = []
for user in users:
    #for product in product_cols:
        #user_preds.append(weights)
    preds.append(weights)
    #user_preds = []

In [48]:
user_preds = []
for product in product_cols:
        user_preds.append(algo.predict(uid,product)[3])


In [51]:
len(user_preds)

24

In [52]:
pred_weight = np.array([user_preds, weights])

In [53]:
pred_weight

array([[5.39327454e-02, 5.39339587e-02, 7.17379605e-01, 5.53066014e-02,
        9.25276088e-02, 6.06877737e-02, 5.83498428e-02, 5.37643589e-02,
        5.37665396e-02, 5.37687230e-02, 5.51289209e-02, 5.37732456e-02,
        7.29315966e-02, 5.40873187e-02, 5.38554217e-02, 7.21926576e-02,
        7.15944816e-02, 5.43430845e-02, 5.39780853e-02, 6.66836940e-02,
        9.70404784e-02, 6.24596118e-02, 5.47486673e-02, 5.40836776e-02],
       [0.00000000e+00, 0.00000000e+00, 2.07031678e-01, 1.97027080e-04,
        4.23389304e-02, 1.20405438e-03, 7.64027234e-03, 4.86000131e-03,
        3.37135226e-03, 1.10116246e-02, 2.37527091e-02, 7.22432628e-04,
        2.66862234e-02, 5.38540686e-03, 8.75675912e-05, 1.80148427e-01,
        1.12984085e-01, 4.59729854e-04, 1.75135182e-04, 6.44059634e-02,
        1.99894919e-01, 1.04095974e-01, 3.48081175e-03, 6.56756934e-05]])

In [55]:
np.average(pred_weight,axis=0)

array([0.02696637, 0.02696698, 0.46220564, 0.02775181, 0.06743327,
       0.03094591, 0.03299506, 0.02931218, 0.02856895, 0.03239017,
       0.03944082, 0.02724784, 0.04980891, 0.02973636, 0.02697149,
       0.12617054, 0.09228928, 0.02740141, 0.02707661, 0.06554483,
       0.1484677 , 0.08327779, 0.02911474, 0.02707468])

In [56]:
user_preds = np.average(pred_weight,axis=0)

In [57]:
np.argsort(user_preds)

array([ 0,  1, 14, 23, 18, 11, 17,  3,  8, 22,  7, 13,  5,  9,  6, 10, 12,
       19,  4, 21, 16, 15, 20,  2], dtype=int64)

In [58]:
len(user_preds)

24

### temp

In [60]:
preds =[]
user_preds = []
for user in users:
    for product in product_cols:
        user_preds.append(algo.predict(user,product)[3])
    pred_weight = np.array([user_preds, weights])
    user_preds = np.average(pred_weight,axis=0)
    preds.append(user_preds)
    user_preds = []

In [19]:
preds =np.array([np.array(predi) for predi in preds])

In [61]:
preds = np.argsort(preds, axis=1)
preds = np.fliplr(preds)

In [62]:
preds[0]

array([ 2, 20, 15, 16, 21,  4, 19, 12, 10,  6,  9,  5, 13,  7, 22,  8,  3,
       17, 11, 18, 23, 14,  1,  0], dtype=int64)

In [63]:
final_month_training_cols = train
final_month_training_cols = final_month_training_cols.drop(product_cols, axis=1)

In [64]:
#Get the target columns
labels_final_month = train[product_cols]
#Add ncodpers to the dataframe
labels_final_month['ncodpers'] = train.ncodpers
labels_final_month = labels_final_month.set_index("ncodpers")
stacked_labels_final_month = labels_final_month.stack()
filtered_labels_final_month = stacked_labels_final_month.reset_index()
filtered_labels_final_month.columns = ["ncodpers", "product", "owned"]
#Only select the rows where there is a new product.
filtered_labels_final_month = filtered_labels_final_month[filtered_labels_final_month["owned"] == 1]


final_month = filtered_labels_final_month.drop_duplicates(keep='last')
labels_final_month_final = final_month['product']
final_month_ncodpers = final_month.ncodpers
#Remove the columns that are not needed to train the model.
final_month = final_month.drop(['ncodpers','owned','product'], axis = 1)

C:\Users\liamc\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [5]:
labels_final_month_final_cat = labels_final_month_final.astype('category').cat.codes

NameError: name 'labels_final_month_final' is not defined

In [66]:
used_products = pd.DataFrame()
used_products['product'] = labels_final_month_final_cat
used_products['ncodpers'] = final_month_ncodpers
used_products = used_products.drop_duplicates(keep = 'last')

In [67]:
#create a dictionary to store each product a customer already has
used_recommendation_products = {}
product_cols = np.array(product_cols)
#iterate through used_products and add each one to used_recommendation_products
for idx,row_val in used_products.iterrows():
    used_recommendation_products.setdefault(row_val['ncodpers'],[]).append(product_cols[row_val['product']])
    if len(used_recommendation_products) % 100000 == 0:
        print(len(used_recommendation_products))

100000
200000
300000
300000
400000
500000
600000
600000
600000
600000
600000
600000


In [28]:
#test_ids are the customer codes for the testing data.
test_ids = np.array(pd.read_csv(r'E:\Santander Recommendation\test_ver2.csv',usecols=['ncodpers'])['ncodpers'])
target_cols_all = np.array(product_cols)
final_preds = []
#iterate through our model's predictions (pred) and add the 7 most recommended products that the customer does not have.
for idx,predicted in enumerate(preds):
    ids = test_ids[idx]
    top_product = target_cols_all[predicted]
    used_products = used_recommendation_products.get(ids,[])
    new_top_product = []
    for product in top_product:
        if product not in used_products:
            new_top_product.append(product)
        if len(new_top_product) == 7:
            break
    final_preds.append(' '.join(new_top_product))
    if len(final_preds) % 100000 == 0:
        print(len(final_preds))

100000
200000
300000
400000
500000
600000
700000
800000
900000


In [68]:
test_ids = np.array(pd.read_csv(r'E:\Santander Recommendation\test_ver2.csv',usecols=['ncodpers'])['ncodpers'])
submission = pd.DataFrame({'ncodpers':test_ids,'added_products':final_preds})

In [69]:
submission = pd.DataFrame({'ncodpers':test_ids,'added_products':final_preds})
submission.to_csv('submission.csv',index=False)

In [70]:
submission.shape

(929615, 2)

In [71]:
submission.head()

,ncodpers,added_products
0,15889,ind_recibo_ult1 ind_nom_pens_ult1 ind_nomina_u...
1,1170544,ind_recibo_ult1 ind_nom_pens_ult1 ind_nomina_u...
2,1170545,ind_recibo_ult1 ind_nom_pens_ult1 ind_nomina_u...
3,1170547,ind_recibo_ult1 ind_nom_pens_ult1 ind_nomina_u...
4,1170548,ind_recibo_ult1 ind_nom_pens_ult1 ind_nomina_u...
